<a href="https://colab.research.google.com/github/FATCK06/ProjectAPI_FirstSemester/blob/BranchPai_BackEnd/RubyFox_Analise_Populacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extraindo dados populacional de São José dos Campos:**

Nosso Github: https://github.com/FATCK06/ProjectAPI_FirstSemester

Desenvolvemos esse código dentro do Google Colab para apresentar dados do censo de IBGE dos anos 2010 e 2022, tais informações apresenta: População residente de SJC, Faixa etária, Densidade demográfica e População por sexo.

# Instalando bibliotecas Python para o progama:

Base de todo programa para funcionar.

In [ ]:
!pip install flask pyngrok matplotlib plotly pandas sidrapy streamlit flask-cors --upgrade

# Inserindo Token Pyngrok:

Importante, não esquecer de rodar isso primeiro.

In [2]:
!ngrok config add-authtoken 32Qi64IHMghRwMNkoFZu49G4xIF_yYjaH7aJoXcfiJEhhhCs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Carregando dados CSV do IBGE:

Verificação via link público e analisar se os dados estão corretos.

In [7]:
import pandas as pd
import requests

# ==============================================================================
# Cole aqui o dicionário de URLs do seu script do dashboard
# ==============================================================================
urls_csv = {
    "idade_sexo_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20Idade%20e%20sexo.csv",
    "pop_residencia_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20popula%C3%A7%C3%A3o%20e%20residencia.csv",
    "pop_residencia_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/populacao_residente_sjc_2010.csv",
    "densidade_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/densidade_demografica_sjc_2010.csv",
    "faixa_mulheres_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_mulheres_2010.csv",
    "faixa_homens_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_homens_2010.csv"
}
# ==============================================================================

print("─" * 60)
print("🚀 Iniciando a verificação de todos os links dos arquivos CSV...")
print("─" * 60)

links_com_falha = 0

# Itera sobre cada nome e url no dicionário
for nome_arquivo, url in urls_csv.items():
    print(f"Verificando '{nome_arquivo}'...")
    print(f"URL: {url}")

    try:
        # Tenta ler apenas as primeiras 5 linhas para ser mais rápido (head)
        # O Pandas precisa de um 'User-Agent' para evitar ser bloqueado (erro 403)
        # Este cabeçalho simula um navegador web.
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
        }
        # Primeiro, verificamos se o link existe (status 200 OK)
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Lança um erro se o status não for 200

        # Se o link estiver OK, tentamos ler com o Pandas
        df_teste = pd.read_csv(url)
        print("✅ SUCESSO: O link é válido e o arquivo CSV foi lido corretamente.")

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ FALHA: Erro de HTTP. O link parece estar quebrado ou inacessível. (Status: {http_err.response.status_code})")
        links_com_falha += 1

    except pd.errors.ParserError as parser_err:
        print(f"❌ FALHA: O link é válido, mas o arquivo não parece ser um CSV bem formatado. (Erro de Parser)")
        links_com_falha += 1

    except Exception as e:
        print(f"❌ FALHA: Ocorreu um erro inesperado. (Erro: {e})")
        links_com_falha += 1

    print("-" * 50)

print("\n" + "─" * 60)
print("🏁 Verificação Concluída!")
if links_com_falha == 0:
    print("🎉 Ótima notícia! Todos os links foram verificados e estão funcionando perfeitamente.")
else:
    print(f"⚠️ Atenção: {links_com_falha} link(s) apresentaram falha. Revise as URLs com problema.")
print("─" * 60)


────────────────────────────────────────────────────────────
🚀 Iniciando a verificação de todos os links dos arquivos CSV...
────────────────────────────────────────────────────────────
Verificando 'idade_sexo_2022'...
URL: https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20Idade%20e%20sexo.csv
✅ SUCESSO: O link é válido e o arquivo CSV foi lido corretamente.
--------------------------------------------------
Verificando 'pop_residencia_2022'...
URL: https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20popula%C3%A7%C3%A3o%20e%20residencia.csv
✅ SUCESSO: O link é válido e o arquivo CSV foi lido corretamente.
--------------------------------------------------
Verificando 'pop_residencia_2010'...
URL: https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/populacao_residente_sjc_2010.csv
✅ 

# Ainda sobre construção, será onde o programa rodará com Flask juntamente com streamlit:

Como dito antes, ainda está em construção então falta inserir mais algumas coisas, como o Flask.

In [4]:
import os
import socket
import subprocess
import time
import atexit
from pyngrok import ngrok

# ==================================================
# 1. Código do Streamlit com carregamento via URL
# ==================================================
# Usamos r"""...""" para criar uma string de múltiplas linhas que será salva no arquivo.
streamlit_code = r"""
import streamlit as st
import pandas as pd
import plotly.express as px
import re

st.set_page_config(
    page_title="Dashboard Populacional SJC",
    page_icon="📊",
    layout="wide"
)

# =========================
# Funções auxiliares
# =========================

@st.cache_data
def limpar_colunas(df):
    cols = df.columns.str.strip().str.lower()
    replacements = {
        ' ': '_', 'ç': 'c', 'ã': 'a', 'õ': 'o', 'á': 'a', 'é': 'e',
        'í': 'i', 'ó': 'o', 'ú': 'u', 'â': 'a', 'ê': 'e', 'î': 'i',
        'ô': 'o', 'û': 'u'
    }
    for old, new in replacements.items():
        cols = cols.str.replace(old, new, regex=False)
    cols = cols.str.replace(r'[^a-z0-9_]', '', regex=True)
    df.columns = cols
    return df

def converter_para_numerico(series):
    if pd.api.types.is_string_dtype(series):
        series = (
            series.str.replace(r"(dados alterados)", "", regex=True)
            .str.replace(r"\\.", "", regex=True)
            .str.replace(r",", ".", regex=True)
        )
    return pd.to_numeric(series, errors='coerce')

@st.cache_data
def carregar_csvs():
    # ==============================================================================
    # <<< ÁREA DE ALTERAÇÃO DOS LINKS >>>
    # Altere os links abaixo para apontar para os seus arquivos CSV no GitHub.
    # Lembre-se de usar o link "Raw" (Bruto) de cada arquivo.
    # Dica: Evite usar caracteres especiais e acentos nos nomes dos arquivos
    #       para facilitar a criação das URLs.
    # ==============================================================================
    urls_csv = {
        "idade_sexo_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20Idade%20e%20sexo.csv",
        "pop_residencia_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20popula%C3%A7%C3%A3o%20e%20residencia.csv",
        "pop_residencia_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/populacao_residente_sjc_2010.csv",
        "densidade_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/densidade_demografica_sjc_2010.csv",
        "faixa_mulheres_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_mulheres_2010.csv",
        "faixa_homens_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_homens_2010.csv"
    }
    # ==============================================================================

    try:
        # Carrega os dados brutos a partir das URLs
        datasets_raw = {key: pd.read_csv(url) for key, url in urls_csv.items()}

        # Cria uma cópia limpa para manipulação
        datasets = {}
        for key, df in datasets_raw.items():
            # Preserva os dados originais de 2010 para busca de strings
            if key in ["pop_residencia_2010", "faixa_homens_2010", "faixa_mulheres_2010"]:
                datasets[key] = df.copy()
            # Limpa os nomes das colunas dos outros
            else:
                datasets[key] = limpar_colunas(df.copy())

        # Converte colunas numéricas dos arquivos de 2022
        for col in datasets["idade_sexo_2022"].columns:
            if col not in ["tipo", "ano"]:
                datasets["idade_sexo_2022"][col] = converter_para_numerico(datasets["idade_sexo_2022"][col])

        for col in datasets["pop_residencia_2022"].columns:
            if col not in ['ano']:
                datasets["pop_residencia_2022"][col] = converter_para_numerico(datasets["pop_residencia_2022"][col])

        return datasets

    except Exception as e:
        st.error(f"Ocorreu um erro ao carregar os dados de uma URL. Verifique os links no código. Erro: {e}")
        st.stop()

def normalizar_faixa_etaria_2010(indicador):
    indicador = str(indicador).lower()
    if 'menos de 1' in indicador or '1 a 4' in indicador: return '0_a_4_anos'
    if '5 a 9' in indicador: return '5_a_9_anos'
    if '10 a 14' in indicador: return '10_a_14_anos'
    if '15 a 19' in indicador: return '15_a_19_anos'
    if '20 a 24' in indicador: return '20_a_24_anos'
    if '25 a 29' in indicador: return '25_a_29_anos'
    if '30 a 34' in indicador: return '30_a_34_anos'
    if '35 a 39' in indicador: return '35_a_39_anos'
    if '40 a 44' in indicador: return '40_a_44_anos'
    if '45 a 49' in indicador: return '45_a_49_anos'
    if '50 a 54' in indicador: return '50_a_54_anos'
    if '55 a 59' in indicador: return '55_a_59_anos'
    if '60 a 64' in indicador: return '60_a_64_anos'
    if '65 a 69' in indicador: return '65_a_69_anos'
    if '70 a 74' in indicador: return '70_a_74_anos'
    if '75 a 79' in indicador: return '75_a_79_anos'
    if '80 a 84' in indicador: return '80_a_84_anos'
    if '85 a 89' in indicador: return '85_a_89_anos'
    if '90 a 94' in indicador: return '90_a_94_anos'
    if '95 a 99' in indicador: return '95_a_99_anos'
    if '100 anos ou mais' in indicador: return '100_anos_ou_mais'
    return None

# =========================
# Título Principal
# =========================
st.title("📊 Análise Populacional - São José dos Campos")
datasets = carregar_csvs()

# --- Extração de Totais ---
total_correto_2022 = datasets["pop_residencia_2022"]["populacao_residente"].iloc[0]
df_pop_2010 = datasets["pop_residencia_2010"]
# Busca pelo valor exato no CSV original, antes da limpeza de colunas
total_correto_2010 = df_pop_2010.loc[df_pop_2010['Indicador'] == 'População residente', 'Valor'].iloc[0]


# ===================================================================
# GRÁFICO 1: POPULAÇÃO POR SEXO & MÉTRICAS
# ===================================================================
st.header("População Geral")
col1, col2 = st.columns([3, 1])

with col1:
    # --- Cálculo 2022 ---
    df_idade_sexo_2022 = datasets["idade_sexo_2022"]
    homens_2022_raw = df_idade_sexo_2022[df_idade_sexo_2022["tipo"].str.contains("homens", na=False, case=False)]["total"].sum()
    mulheres_2022_raw = df_idade_sexo_2022[df_idade_sexo_2022["tipo"].str.contains("mulheres", na=False, case=False)]["total"].sum()
    soma_raw_2022 = homens_2022_raw + mulheres_2022_raw
    homens_2022 = (homens_2022_raw / soma_raw_2022) * total_correto_2022 if soma_raw_2022 > 0 else 0
    mulheres_2022 = (mulheres_2022_raw / soma_raw_2022) * total_correto_2022 if soma_raw_2022 > 0 else 0
    df_sexo_2022 = pd.DataFrame([{"Sexo": "Homens", "População": homens_2022, "Ano": "2022"}, {"Sexo": "Mulheres", "População": mulheres_2022, "Ano": "2022"}])

    # --- Cálculo 2010 ---
    homens_2010_raw = datasets["faixa_homens_2010"]['Valor'].sum()
    mulheres_2010_raw = datasets["faixa_mulheres_2010"]['Valor'].sum()
    soma_raw_2010 = homens_2010_raw + mulheres_2010_raw
    homens_2010 = (homens_2010_raw / soma_raw_2010) * total_correto_2010 if soma_raw_2010 > 0 else 0
    mulheres_2010 = (mulheres_2010_raw / soma_raw_2010) * total_correto_2010 if soma_raw_2010 > 0 else 0
    df_sexo_2010 = pd.DataFrame([{"Sexo": "Homens", "População": homens_2010, "Ano": "2010"}, {"Sexo": "Mulheres", "População": mulheres_2010, "Ano": "2010"}])

    df_sexo_final = pd.concat([df_sexo_2010, df_sexo_2022], ignore_index=True)
    fig1 = px.bar(df_sexo_final, x="Sexo", y="População", color="Ano", barmode="group", title="População por Sexo (2010 vs 2022)", text_auto='.3s')
    fig1.update_traces(textangle=0, textposition="outside")
    st.plotly_chart(fig1, use_container_width=True)

with col2:
    st.metric("População Total 2022", f"{int(total_correto_2022):,}".replace(",", "."))
    st.metric("População Total 2010", f"{int(total_correto_2010):,}".replace(",", "."))
    if total_correto_2010 > 0:
        st.metric("Crescimento Populacional", f"{(total_correto_2022/total_correto_2010 - 1):.2%}")

# ===================================================================
# GRÁFICO 2: DENSIDADE DEMOGRÁFICA
# ===================================================================
st.header("Densidade Demográfica")
# CORREÇÃO: Os nomes das colunas de densidade são diferentes, então precisamos tratar isso.
df_densidade_2010 = limpar_colunas(datasets["densidade_2010"].copy())

data_densidade = []
if "densidade_demografica_habkm2" in df_densidade_2010.columns:
    densidade_2010 = df_densidade_2010["densidade_demografica_habkm2"].iloc[0]
    data_densidade.append({"Ano": "2010", "Densidade (hab/km²)": densidade_2010})
if "densidade" in datasets["pop_residencia_2022"].columns:
    densidade_2022 = datasets["pop_residencia_2022"]["densidade"].iloc[0]
    data_densidade.append({"Ano": "2022", "Densidade (hab/km²)": densidade_2022})

if data_densidade:
    df_densidade = pd.DataFrame(data_densidade)
    fig2 = px.line(df_densidade, x="Ano", y="Densidade (hab/km²)", markers=True, title="Densidade Demográfica (2010 vs 2022)")
    fig2.update_traces(text=df_densidade["Densidade (hab/km²)"].round(2), textposition="top center")
    st.plotly_chart(fig2, use_container_width=True)
else:
    st.warning("Dados de densidade demográfica não disponíveis.")


# ===================================================================
# GRÁFICO 3: FAIXA ETÁRIA (2010 vs 2022)
# ===================================================================
st.header("Distribuição da População por Faixa Etária")
# --- Cálculo 2022 ---
faixas_2022 = [c for c in df_idade_sexo_2022.columns if c not in ["ano", "tipo", "total"]]
faixa_total_2022_raw = df_idade_sexo_2022.loc[df_idade_sexo_2022["tipo"].str.lower().isin(["homens", "mulheres"]), faixas_2022].sum().fillna(0)
fator_2022 = total_correto_2022 / faixa_total_2022_raw.sum() if faixa_total_2022_raw.sum() > 0 else 1
faixa_total_2022 = (faixa_total_2022_raw * fator_2022).reset_index(); faixa_total_2022.columns = ["faixa_etaria", "populacao"]; faixa_total_2022["ano"] = "2022"

# --- Cálculo 2010 ---
df_h_2010 = datasets["faixa_homens_2010"]; df_m_2010 = datasets["faixa_mulheres_2010"]
df_faixa_2010_combined = pd.concat([df_h_2010, df_m_2010])
df_faixa_2010_combined['faixa_normalizada'] = df_faixa_2010_combined['Indicador'].apply(normalizar_faixa_etaria_2010)
df_faixa_2010_combined = df_faixa_2010_combined.dropna(subset=['faixa_normalizada']) # Remove linhas que não foram mapeadas
faixa_total_2010_raw = df_faixa_2010_combined.groupby('faixa_normalizada')['Valor'].sum()
fator_2010 = total_correto_2010 / faixa_total_2010_raw.sum() if faixa_total_2010_raw.sum() > 0 else 1
faixa_total_2010 = (faixa_total_2010_raw * fator_2010).reset_index(); faixa_total_2010.columns = ["faixa_etaria", "populacao"]; faixa_total_2010["ano"] = "2010"

# --- Junção e Plot ---
faixa_df_final = pd.concat([faixa_total_2010, faixa_total_2022], ignore_index=True)
ordem_faixas = sorted(faixa_df_final['faixa_etaria'].unique(), key=lambda x: int(re.search(r'\d+', x).group(0)))
faixa_df_final['faixa_etaria_display'] = faixa_df_final['faixa_etaria'].str.replace('_', ' ').str.replace(' a ', '-').str.replace(' anos', '').str.replace(' ou mais', '+')
ordem_display = [d.replace('_', ' ').replace(' a ', '-').replace(' anos', '').replace(' ou mais', '+') for d in ordem_faixas]

if not faixa_df_final.empty:
    fig3 = px.bar(faixa_df_final, x="faixa_etaria_display", y="populacao", color="ano", barmode="group", title="População por Faixa Etária (2010 vs 2022)",
                  category_orders={"faixa_etaria_display": ordem_display},
                  labels={"faixa_etaria_display": "Faixa Etária", "populacao": "População"},
                  text_auto='.3s')
    fig3.update_xaxes(tickangle=45)
    fig3.update_traces(textangle=0, textposition="outside")
    st.plotly_chart(fig3, use_container_width=True)
else:
    st.warning("Dados de faixa etária não disponíveis para exibição.")
"""

# ==================================================
# 2. Salva o código em um arquivo .py
# ==================================================
STREAMLIT_APP_FILE = "app_dashboard.py"
with open(STREAMLIT_APP_FILE, "w", encoding="utf-8") as f:
    f.write(streamlit_code)
print(f"✅ Código do Streamlit salvo em '{STREAMLIT_APP_FILE}'")

# ==================================================
# 3. Executa o Streamlit e o ngrok
# ==================================================
def is_port_in_use(port: int) -> bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(("localhost", port)) == 0

# <<< IMPORTANTE: Insira seu token do ngrok abaixo >>>
# É uma boa prática não deixar tokens expostos diretamente no código,
# mas para o ambiente do Colab, esta é a forma mais direta.
NGROK_AUTH_TOKEN = "32Qi64IHMghRwMNkoFZu49G4xIF_yYjaH7aJoXcfiJEhhhCs"
if not NGROK_AUTH_TOKEN or NGROK_AUTH_TOKEN == "SEU_TOKEN_AQUI":
    print("⚠️ ATENÇÃO: Insira seu token de autenticação do ngrok para continuar.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    PORTA_STREAMLIT = 8501
    print("🚀 Iniciando o Streamlit em background...")
    proc = subprocess.Popen(
        ["streamlit", "run", STREAMLIT_APP_FILE, "--server.port", str(PORTA_STREAMLIT), "--server.headless", "true", "--server.enableCORS", "false"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding="utf-8"
    )
    atexit.register(proc.kill)
    print("⏳ Aguardando o servidor do Streamlit ficar pronto...")
    start_time = time.time()
    while not is_port_in_use(PORTA_STREAMLIT):
        time.sleep(0.5)
        if time.time() - start_time > 60:
            print("❌ Erro: O Streamlit demorou muito para iniciar.", proc.stderr.read())
            exit()
    print("✅ Servidor Streamlit iniciado com sucesso!")
    try:
        public_url = ngrok.connect(PORTA_STREAMLIT)
        print("─" * 50); print(f"🎉 Seu dashboard está no ar!"); print(f"🔗 URL Pública: {public_url}"); print("─" * 50)
    except Exception as e:
        print(f"❌ Erro ao conectar com o ngrok: {e}")


✅ Código do Streamlit salvo em 'app_dashboard.py'
🚀 Iniciando o Streamlit em background...
⏳ Aguardando o servidor do Streamlit ficar pronto...
✅ Servidor Streamlit iniciado com sucesso!
──────────────────────────────────────────────────
🎉 Seu dashboard está no ar!
🔗 URL Pública: NgrokTunnel: "https://8fbd4d42f95a.ngrok-free.app" -> "http://localhost:8501"
──────────────────────────────────────────────────


# Geração do mapa interativo da cidade de São José dos Campos:

Ainda em construção, aqui será onde o mapa interativo de SJC será inserida no Google Colab.

In [ ]:
!pip install --upgrade pip shapely
!pip install geopandas rtree mapclassify
!pip install --upgrade shapely geopandas rtree mapclassify
!pip install geobr

